In [1]:
# make sure you are on the branch 'topquadrant-shacl' until PR https://github.com/NREL/BuildingMOTIF/pull/308 is merged
from rdflib import Namespace
from buildingmotif import BuildingMOTIF
from buildingmotif.dataclasses import Library, Model

In [2]:
# setup our buildingmotif instance (in-memory for now)
bm = BuildingMOTIF("sqlite://", shacl_engine='topquadrant')

In [3]:
# import 223P ontology from models.open223.info
s223 = Library.load(ontology_graph="libraries/ashrae/223p/ontology/223p.ttl")

2024-09-19 08:59:34,736 | root |  WARNING: An ontology could not resolve a dependency on http://data.ashrae.org/standard223/1.0/extensions/settings (No row was found when one was required). Check this is loaded into BuildingMOTIF
2024-09-19 08:59:34,736 | root |  WARNING: An ontology could not resolve a dependency on http://data.ashrae.org/standard223/1.0/model/core (No row was found when one was required). Check this is loaded into BuildingMOTIF
2024-09-19 08:59:34,737 | root |  WARNING: An ontology could not resolve a dependency on http://data.ashrae.org/standard223/1.0/model/equipment (No row was found when one was required). Check this is loaded into BuildingMOTIF
2024-09-19 08:59:34,738 | root |  WARNING: An ontology could not resolve a dependency on http://data.ashrae.org/standard223/1.0/vocab/enumeration (No row was found when one was required). Check this is loaded into BuildingMOTIF
2024-09-19 08:59:34,738 | root |  WARNING: An ontology could not resolve a dependency on http:/

2024-07-02 17:26:41,367 | root |  WARNING: An ontology could not resolve a dependency on http://qudt.org/2.1/vocab/currency (No row was found when one was required). Check this is loaded into BuildingMOTIF


2024-07-02 17:26:41,367 | root |  WARNING: An ontology could not resolve a dependency on http://qudt.org/2.1/vocab/unit (No row was found when one was required). Check this is loaded into BuildingMOTIF


2024-07-02 17:26:41,368 | root |  WARNING: An ontology could not resolve a dependency on http://www.w3.org/ns/shacl# (No row was found when one was required). Check this is loaded into BuildingMOTIF


In [4]:
# create a model to hold an example building from models.open223.info
nrel_example_building = Model.create(Namespace("urn:nrel"))
nrel_example_building.graph.parse("https://models.open223.info/nrel-example.ttl")

<Graph identifier=ea8b61bc-6562-4325-b77e-0202ff416aa6 (<class 'rdflib.graph.Graph'>)>

In [ ]:
# find any issues with the model w.r.t. 223P ontology (you'll need Java installed for the "fast" validation with the topquadrant engine)
import time
start = time.time()
ctx = nrel_example_building.validate([s223.get_shape_collection()], error_on_missing_imports=False)
print(f"Performed validation in {time.time() - start} seconds")
ctx.valid

2024-09-19 08:59:38,035 | buildingmotif.dataclasses.shape_collection |  WARNING: Could not resolve import of http://data.ashrae.org/standard223/1.0/extensions/settings from Libraries (No row was found when one was required). Trying shape collections
2024-09-19 08:59:38,038 | buildingmotif.dataclasses.shape_collection |  WARNING: Could not resolve import of http://data.ashrae.org/standard223/1.0/extensions/settings from Libraries. Trying shape collections
2024-09-19 08:59:38,039 | buildingmotif.dataclasses.shape_collection |  WARNING: Could not resolve import of http://www.w3.org/ns/shacl# from Libraries (No row was found when one was required). Trying shape collections
2024-09-19 08:59:38,041 | buildingmotif.dataclasses.shape_collection |  WARNING: Could not resolve import of http://www.w3.org/ns/shacl# from Libraries. Trying shape collections
2024-09-19 08:59:38,041 | buildingmotif.dataclasses.shape_collection |  WARNING: Could not resolve import of http://data.ashrae.org/standard223/

2024-07-02 17:26:46,836 | buildingmotif.dataclasses.shape_collection |  WARNING: Could not resolve import of http://data.ashrae.org/standard223/1.0/extensions/settings from Libraries (No row was found when one was required). Trying shape collections


2024-07-02 17:26:46,838 | buildingmotif.dataclasses.shape_collection |  WARNING: Could not resolve import of http://data.ashrae.org/standard223/1.0/extensions/settings from Libraries. Trying shape collections


2024-07-02 17:26:46,838 | buildingmotif.dataclasses.shape_collection |  WARNING: Could not resolve import of http://data.ashrae.org/standard223/1.0/vocab/enumeration from Libraries (No row was found when one was required). Trying shape collections


2024-07-02 17:26:46,840 | buildingmotif.dataclasses.shape_collection |  WARNING: Could not resolve import of http://data.ashrae.org/standard223/1.0/vocab/enumeration from Libraries. Trying shape collections


Performed validation in 71.6988320350647 seconds


0

In [ ]:
# list out issues with specific nodes (available severities are Violation, Warning, and Info)
for focus_node, diffs in ctx.get_reasons_with_severity("Violation").items():
    if len(diffs) == 0:
        continue
    print(focus_node)
    for diff in diffs:
        print("  - " + diff.reason())

In [ ]:
# print the whole validation report
print(ctx.report_string)